In [1]:
import re
import os
import numpy as np
import pandas as pd

In [3]:
def extract_records(path:str, split_input=False)-> list:
    with open(path) as f:
        raw = f.read()



    raw_splitted = raw.split('\t')


    # questo ciclo while serve perchè il formato del file non è ben diviso dagli \t, così vengono inseriti gli \t mancanti tra la classe della riga precedente, e l'id della successiva
    # per funzionare serve rimuovere manualmente dal file tsv la riga degli header.
    j = 3
    splitted = []
    splitted += raw_splitted[0:3]
    while j < len(raw_splitted):
        if len(raw_splitted[j]) in range(5,30):
            raw_splitted[j] = re.sub(r"(^[01])",r'\1\t',raw_splitted[j]) # aggiungo \t per poi subito dopo separare
            split = raw_splitted[j].split('\t')
        else:
            split = [raw_splitted[j]]

        splitted += split
        j+=1

        

    res_list = []


    # se voglio che il prompt e il target siano trattati come unico testo
    if not split_input:
        i = 0
        while i < len(splitted):
            text = splitted[i+1].strip('\"') + '\n' + splitted[i+2].strip('\"')
            id = splitted[i]
            class_ = splitted[i+3]
            res_list.append({
                'id':id.strip('\n'),
                'text':text,
                'class': int(class_.strip('\n'))
            })
            i+=4

    # se voglio che il prompt e il target siano tenuti separati
    if split_input:
        i = 0
        while i < len(splitted):
            prompt = splitted[i+1]
            target = splitted[i+2]
            id = splitted[i]
            class_ = splitted[i+3]
            res_list.append({
                'id':id.strip('\n'),
                'prompt': prompt.strip('\"'),
                'target': target.strip('\"'),
                'class': int(class_.strip('\n'))
            })
            i+=4

    

    return res_list




train_ted_splitted = extract_records('../data/discotex_1_ted_train.tsv', True)
train_ted = extract_records('../data/discotex_1_ted_train.tsv', False)

"""
DATASET WIKI NON UTILIZZATO ALLA FINE
train_wiki_splitted = extract_records('../data/discotex_1_wiki_train.tsv', True)
train_wiki = extract_records('../data/discotex_1_wiki_train.tsv', False)
"""

test_ted_splitted = extract_records('../data/discotex_1_ted_test_set_release.tsv', True)
test_ted = extract_records('../data/discotex_1_ted_test_set_release.tsv', False)

"""
DATASET WIKI NON UTILIZZATO ALLA FINE
test_wiki_splitted = extract_records('../data/discotex_1_wiki_test_set_release.tsv', True)
test_wiki = extract_records('../data/discotex_1_wiki_test_set_release.tsv', False)
"""

Ora faccio qualche controllo per capire se tutto ha preservato il corretto ordine

In [62]:
# controllo le dimensioni per capire se tutto quadra
len(train_ted_splitted),len(train_ted)

(8000, 8000)

In [63]:
len(test_ted_splitted),len(test_ted)

(800, 800)

Ora creo le cartelle e i file

In [4]:
def print_files(train:list, test:list, dataset_name:str, splitted=False):
    """
    Funzione che stampa i documenti come singoli file di testo, il nome del file conterrà l'id e la label.
    Se "splitted==True", prompt e target saranno stampati su file diversi, sempre nel nome del file quindi sarà presente o "prompt" o "target"
    
    """

    
    dir = f'../data/{dataset_name}'
    train_dir = f'../data/{dataset_name}/train'
    test_dir = f'../data/{dataset_name}/test'

    try:
        os.makedirs(dir)
        os.makedirs(train_dir)
        os.makedirs(test_dir)
    except FileExistsError:
        pass

    if not splitted:
        for i,record in enumerate(train):
            with open(f'{train_dir}/{i}_{record["class"]}.txt', 'w') as f:
                f.write(record['text'])

        for i,record in enumerate(test):
            with open(f'{test_dir}/{i}_{record["class"]}.txt', 'w') as f:
                f.write(record['text'])

    else:
        for i,record in enumerate(train):
            with open(f'{train_dir}/{i}_{record["class"]}_prompt.txt', 'w') as f:
                f.write(record['prompt'])
            with open(f'{train_dir}/{i}_{record["class"]}_target.txt', 'w') as f:
                f.write(record['target'])

        for i,record in enumerate(test):
            with open(f'{test_dir}/{i}_{record["class"]}_prompt.txt', 'w') as f:
                f.write(record['prompt'])
            with open(f'{test_dir}/{i}_{record["class"]}_target.txt', 'w') as f:
                f.write(record['target'])




print_files(train_ted, test_ted, 'TED')
print_files(train_ted_splitted, test_ted_splitted, 'TED_split', True)

"""
DATASET WIKI NON UTILIZZATO ALLA FINE
print_files(train_wiki, test_wiki, 'WIKI')
print_files(train_wiki_splitted, test_wiki_splitted, 'WIKI_split', True)
"""
